In [19]:
import pandas as pd
import sys
import os
import uuid
import time

sys.path.append("../../")

#Import anthropic for llm generation
from anthropic import Anthropic
#Future AGI Client for logging model data 
from fi.client import Client
from fi.client import ModelTypes, Environments

**LOAD ENV VARS**

In [4]:
os.environ["ANTHROPIC_API_KEY"]="ENTER-YOUR-KEY"

In [20]:
#pandas setting
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 2000)
pd.set_option('display.max_colwidth', None)

**Load Dataset**

In [21]:
data_df = pd.read_excel("Oliv_data.xlsx", sheet_name="Sheet1")

In [22]:
data_df = data_df[data_df['Prompt'].notna()]

**Initializing FI Client**

In [10]:

fi_client = Client(
    api_key="8e54ce886a6143a4b5845069e15ff4da",
    secret_key="11d0a3d5b4284c239b625db849d3e5a0"
  
)

In [16]:
def upload_data(pt_data):
    """
    Function for logging data into model
    """
    return fi_client.log(
  "oliv-prompt-template-test-v8",
  ModelTypes.GENERATIVE_LLM,
  Environments.PRODUCTION,
  "v1",
  # str(uuid.uuid4()),
  int(time.time()),
  {
    "chat_history": pt_data
  }
).result()

In [23]:
prompt_template = """I am going to show you a transcript between participants from selling team 
$selling_company$ and buying team $buying_company$. 
$selling_company$ is the selling company and $buying_company$ is the buying company. 
Come up with answers to a set of questions. 
I will first provide a sample transcript, list of participants for the sample transcript, and for each question I will provide the expected output for the sample transcript. 
Then I will provide you with the actual meeting participants list and the actual transcript below it. 
The actual transcript language code is $language_code$.
$sample_participants$
$sample_transcript$
$questions$
$instructions$
$output_schema$
Actual meeting data starts here ---
$actual_participants$
$actual_transcript$
"""

In [17]:
for idx,row in data_df.iterrows():
        data_list =[]
        if row["Prompt"].startswith("I am going to show you a transcript between participants"):
            variables={"selling_company":"Oliv",
                         "buying_company":"Iti-digital",
                         "language_code":"en_us",
                         "sample_participants":"""Sample meeting participants:
    Selling team: Adam Janbolat, Ali Chaichy
    Buying team: Murali Allada, Gabriella, John Handra, Guru, David Cheng, Sekhar Boddu, Michaela Dawson""",
                        "sample_transcript":""" Sample transcript: Adam Janbolat : first off, before I forget, I send you that excel sheet Murali Allada : for the sizing Adam Janbolat : and all that. Excellent. I Murali Allada : did get that. Yeah, it's exactly the same as the original sizing. Murali Allada : I Adam Janbolat : take those numbers and plug it into our calculator. And that's how we come up with the pricing. I don't want to say spend a lot of time on it. It can be good estimations, like we've done in the past. Maybe add a little bit of growth in there, a little bit of. What do you call it? Like, how do you feel? Like about the growth? So I mean, don't be conservative. Which is Murali Allada : it's Adam Janbolat : really easy for us to say, don't be conservative, right? But I mean. We usually factor in 20%, 30%, whatever. And your case here before we did, like, 30%, and we ramped it up month after month. So anyway, just let us know whichever way you want to do that. Adam Janbolat : Email a few more people, wait for a couple of more minutes to make sure that everybody's on the call will do a quick set of introductions. And then we'll just start the conversation. And like I said, morality today. I think I talked to Ali earlier about it. Also. Happy to show a quick demo how to create the vector database. All that. But I would love to hear from you guys. Some of the use cases you're thinking about. You know what I mean? What are you thinking about in terms of generative AI. How can we help you expedite that process. Doing a POC or whatever. And then I'll go through some of the assets that we have on the website today as well. But. This is going to be a 30 minutes call. It's going to be more of a discovery call, quick demo. And then if we need to dig deeper, we can have another 1 hour call coming up in next couple of weeks. To really define the use case and start working on the POC. Murali Allada : So I can give you some examples. Adam, we're looking at it from. A very different perspective. Different teams have different requirements as the infrastructure team we are trying. To use gentle AI for discovery purposes. Basically, we've built an internal developer platform, and we want to make that easy to use for developers. So that's one perspective we are looking at that that's more of a documents search case. Basically. Alet Grew. Or maybe Gabriel, talk about analytics, data analytics. How that might be useful for us. Murali Allada : Sounds good. Let's talk it for next week. Adam Janbolat : Maybe for next week. Okay. Any good time for anybody like middle of week? First of all, I know Monday. I don't think anybody wants to be meeting on Monday. Everybody's busy Tuesday. Maybe you Guru : guys sell us. Middle to late part of next week should be good. I think morally, if Murali Allada : you can coordinate that, let's take Adam Janbolat : it from Yes. Okay. That sounds great. Thank you Adam Janbolat : Great, thank you. So I just want to make sure I got the second use case Analytics, the supply chain perspective. You think maybe for demand planning. When new items coming into the network. What would you like to see there? How do you envision using generative AI to help you with some of that. Adam Janbolat : Now if you tell me Hey. We feel like there is a POC. After this call. Maybe. We can document some of the use cases that we talked about. I might be able. To talk internally with some folks here and see hey, if we can maybe allocate some extra funds so you're not spending your own money on doing your test. Sounds good. So here's what I'll do again. I'll be sending some information here. Via an email. Some assets. I'll schedule another call with you. You tell me it sounds to me like sakar and morali. There is interest we can maybe do Friday or early next week. Whatever you want to do, we'll allocate an hour. And also I wanted to bring one other resource to the conversation. Is that reasonable morale. """,
                        "questions":""" Question: Come up with specific next steps that are actionable immediately or within a specific time for the buying team in 10 words or less. Each item should be prefixed by the task owner name followed by a hyphen. Ideally If there is some decent amount of transcript, these should not be empty. Also do 'not' add the assigned to text at the end Sample output: { "buying-team-next-steps" : [ "Murali Allada - Provide feedback on use cases for generative AI", "Murali Allada - Provide examples of requirements for different teams", "Murali Allada - Coordinate next meeting for next week" ] } Question: Come up with key takeaways that are meaningful for the buying team only in 10 words or less. Key takeaway for buying team should exclude insights about the buying team and key takeaways about the selling team that are not relevant for the buying team. Ideally if there is some decent amount of transcript, these should not be empty Sample output: { "buying-team-key-takeaways" : [ "Adam is a Data Architect at Datastax and Ali is a member of the Streaming Sales team", "Vector search is an additional feature of the paid product, Serverless Database", "The Quite Platform is currently utilizing serverless database and streaming", "Adam demonstrated a chatbot built on Astra DB and its codebase in a Jupyter notebook", "Quite team has access and can test AstraDB from their current account" ] } Question: Come up with specific next steps that are actionable immediately or within a specific time for the selling team in 10 words or less. Each item should be prefixed by the task owner name followed by a hyphen. Ideally if there is some decent amount of transcript, these should not be empty. Also do 'not' add the assigned to text at the end. In case there are no members from internal team give a one line short reason for it like 'No member joined from <company-name>' or some other relevant reason as per the situation and here <company-name> should be replaced by the selling company name. Sample output: { "selling-team-next-steps" : [ "Adam Janbolat - Send Excel sheet for sizing", "Adam Janbolat - Email all the links discussed in the call", "Adam Janbolat - Arrange next week's call with Murali and include an additional participant", "Adam Janbolat - Record Quite Platform use cases and discuss credit allocation for Vector DB trial testing with Ali and the internal team" ] } Question: Come up with key takeaways that are meaningful for the selling team only in 10 words or less. Key takeaway for selling team should include meaningful insights about the overall call and about the buying team that may include key takeaways that are not relevant for the buying team. Include topics like key pain points, objections, security and privacy concerns, key personnel to the deals, key events and timeline for the buying team. Ideally If there is some decent amount of transcript, these should not be empty. In case there are no insightful key takeaways for the selling team then use the buying-team-key-takeaways as reference and paraphrase it in a way that it is still useful for the selling team. Sample output: { "selling-team-key-takeaways" : [ "Guru, John, Gabriella, Sekhar Boddu, and Michaela hold key roles in data management, security, and platform technology at Quite", "Quite's infrastructure team uses generative AI for discovery, internal development, and document search", "The supply chain team at Quite is exploring gen AI for demand planning, forecasting, and customer profiling", "Gabriela focuses on identifying root causes for stock shortages", "Future use cases include tracking services and transportation insights, with a focus on data privacy" ] }  """,
                        "instructions":""" Go through the output schema mentioned below and for every field use the description value as guideline for how to extract the field and keep track of the values as you go through the transcript and return them in the format above """,
                        "output_schema":""" --Output Schema-- The output must be a JSON object that strictly conforms to the following JSON schema and do not output the schema itself. { "type" : "object", "properties" : { "buying-team-next-steps" : { "type" : "array", "items" : { "type" : "string" }, "description" : "Come up with specific next steps that are actionable immediately or within a specific time for the buying team in 10 words or less. Each item should be prefixed by the task owner name followed by a hyphen. Ideally If there is some decent amount of transcript, these should not be empty. Also do 'not' add the assigned to text at the end" }, "buying-team-key-takeaways" : { "type" : "array", "items" : { "type" : "string" }, "description" : "Come up with key takeaways that are meaningful for the buying team only in 10 words or less. Key takeaway for buying team should exclude insights about the buying team and key takeaways about the selling team that are not relevant for the buying team. Ideally if there is some decent amount of transcript, these should not be empty" }, "selling-team-next-steps" : { "type" : "array", "items" : { "type" : "string" }, "description" : "Come up with specific next steps that are actionable immediately or within a specific time for the selling team in 10 words or less. Each item should be prefixed by the task owner name followed by a hyphen. Ideally if there is some decent amount of transcript, these should not be empty. Also do 'not' add the assigned to text at the end. In case there are no members from internal team give a one line short reason for it like 'No member joined from <company-name>' or some other relevant reason as per the situation and here <company-name> should be replaced by the selling company name." }, "selling-team-key-takeaways" : { "type" : "array", "items" : { "type" : "string" }, "description" : "Come up with key takeaways that are meaningful for the selling team only in 10 words or less. Key takeaway for selling team should include meaningful insights about the overall call and about the buying team that may include key takeaways that are not relevant for the buying team. Include topics like key pain points, objections, security and privacy concerns, key personnel to the deals, key events and timeline for the buying team. Ideally If there is some decent amount of transcript, these should not be empty. In case there are no insightful key takeaways for the selling team then use the buying-team-key-takeaways as reference and paraphrase it in a way that it is still useful for the selling team." } }, "description" : "Categorized minutes of a meeting between a selling team and prospective buyer(s)." } -- End output schema -- """,
                        "actual_participants":""" --- Actual meeting participants: Selling team: Blake Adams Buying team: Monica Hall  """,
                        "actual_transcript":""" Actual transcript: Blake Adams : Differentiate unity to close. So not so much top of funnel but more mid funnel around meeting, execution, process adherence, CRM, data completion, things like that. So the way that we then bucket is around the actual sale or the prospect lead customer facing interaction, then the AI comes alongside the seller to help them show up fully prepared with all the information they need on the call, take notes and ensure that they get full qualification criteria. If you guys are a medic or bant or spice, whatever methodology you're using, making sure that you optimize the time, get to a yes or a no quickly and then post call culminates all of that data and can help write wrap up emails, push it back into your CRM, update necessary tools in a consistent codified fashion that also aligns to your methodology. So that's it. Very, very high level. Happy to show you around the tool and dive deeper into any of that, but that if I had to give you a 32nd overview of what all of does that tip? Monica Hall : Okay. And I'm sure I asked the question before I took the meeting, but maybe I didn't. Integration into HubSpot seamless. No problem. Blake Adams : We have a native integration. We actually use HubSpot internally and obviously we live and so great integration with HubSpot. Monica Hall : Okay. And with it. So like 90% of my, no, 99.9% of my opportunities and my sales team's opportunities are from virtual meets, just like we're having here. Right. So does Oliv take that and summarize it, put the notes into HubSpot. Can you show me what that looks like? That's a tool that I'm really looking. Yes. Blake Adams : I think you're speaking our language. So what, essentially it starts with a calendar integration, right. So it's going to see your reps, calendar, my instances. I'm a sales leader, so I've got internal meetings, interviews, all the different things. Right. So I've got workflows. But essentially our product team, during onboarding, would work with you to say for your sellers, for your CS team, for your leadership, whatever that looks like, the different types of meetings you execute, we then understand a workflow based on that. So, for instance, with our conversation, Oliv knows it did all of this, but I could change it if I needed to. Hey, this is an external meeting. We're doing discovery. So I'm going to give Blake all the things he needs for a discovery call. So in that it's bucketed in those same three areas that I mentioned, pre call, during and post, essentially in the pre call motion. Everything you see here again is my instance and my configuration, it's all completely customizable. But basically, again, Oliv learned my process and so it knows. Hey, for a first call, Blake, he typically would go to the company LinkedIn, go to the news, go to the website, look at their careers page. Maybe for Monica, he would go to her LinkedIn, check out previous roles, things like that. It then does all of this for me. Gives me a slack message, a teams message, a Google chat, whatever, ten to 20 minutes before the call, hey, you're about to meet with Monica. Here's your dossier. So you walk in prepared, you can send your warm up emails. All that's integrated back then. Any relevant information it learns can be written to HubSpot from there. Then during the call, we would map this scorecard and these fields to your HubSpot instance. Whatever is existing in your HubSpot that you want sellers filling out during a discovery call or afterwards, we can map in on the call. Again, what Oliv is doing is actually transcribing what we say in real time and reading it back with AI. So it's a little more real time than like a read AI. Fireflies, Otter, you know, some of those tools that are just recording, it's trying to actually help the seller drive. So it actually picked up HubSpot, like, right. It heard you say that. It flipped that for me. Now I'll show afterwards anything else that you said. I could use this as a talk track to actually guide me through and say, hey, I need to ask these questions to understand the tech stack. So if you're ramping reps and things like that, you can put this in front of them. Most reps live on this scorecard view because they just like to interact with the prospect. But it's been very easy to understand what I have and haven't uncovered. Then post call, I'm going, well, let's do a few just so I can show you. So are you guys a Google or a Microsoft shop? Monica Hall : I guess Google. We use Google, yeah, Google. Blake Adams : Okay, perfect. Monica Hall : We use Gmail, right? Yes, Google. Blake Adams : Google calendar. Monica Hall : HubSpot. Okay. Blake Adams : HubSpot. And then do you do your meetings because you're Google on Google Meet, or do you do zoom? Primarily zoom? Okay, and then, so you're on Google, you use zoom. And the only other one is chat. So do you guys use Google chat or slack? Monica Hall : Slack. Blake Adams : Okay. So again, just for role play sake, I'm going to have it summarized. Those would be everything that you saw in there could be customized to your sales motion. All of this would happen automatically. Oliv would get kicked out of the zoom and it would know the meeting was over. It would summarize on behalf of the reps, and then this workflow that you see is configurable to what you again, very similar to the prep, what you want your reps doing after a call. So for me, I want to see my takeaways and yours. I want to send a wrap up email. So I've taught all of my templates. It will then actually, once this summary, okay, so here comes the summary. Once this is done, it will help me write the email so I could actually let Oliv compose with my template and input the necessary things that it learned from the conversation that may be relevant. And it's also pretty smart. So like, it knows this is the first conversation. It's giving you high level information. If you had said soc two or non disclosure or anything like that, it would recommend I attach those types of documents. So it's listening for context there. And then this is where you would update your CRM bidirectionally. And this is what I wanted to show you. So it will show the rep what it has uncovered, categorized, and contextually understood based on the conversation, and changed with this nice little green Oliv. But they still can, sanity check, review whatever they want to, if they want to change anything they can. And then the minute they hit next, all of this is written into HubSpot on the opportunity in the corresponding fields. So it's essentially then guiding them step by step, making recommendations based on the context of the discovery call on how to update the CRM accurately. Monica Hall : Okay. And when it updates in the HubSpot CRM, you said it attaches it to the deal or company or you tell us. Blake Adams : So our team would work with you or your revenue operations, whomever, internally. We would then configure where to write, how to write based on what's best for your sales motion. Okay, so for our instance, it writes on the deal, right? But it could write company level contact. It can attach, we can train the model to do whatever you want it to do, essentially. Right. Okay, so that's the reps experience. Now, again, as a sales leader, you have a lot of visibility because it has access to all this data for your whole team. So just to give you, it houses all the deals, reads them from HubSpot, and then can help culminate info really quickly. It'll give you a one pager as a manager for your deal reviews, for your forecasting, for your pipeline, for your coaching, like where reps need support, because it has access to all of the data and you can really easily come in and say, you know, how, how much do we know? Is this a real deal? How much do we need to know? Maybe then you want to game plan with your rep so you can jump in, do mentions, anything like that that you want to. You can also review all of the call recordings from here, make notes, anything like that. Um, I'm trying to think if there's any other, anything else you want to know while I, while we have, because we have plenty of time. So anything else you want to dive into or explore. Monica Hall : Except I jumped off a call with my boss to jump onto this one and I told her I would come right back. But, um, so what I want to do is say, this has potential. Do you have it set up? Like, is it by seat or do you buy the platform? It's per person. Blake Adams : It's really standard SAS model per user per year and then a platform fee. So cost is variable really dependent upon integrations and configure, like customization. So how configurable do you need the instance and how many integrations do we have to build? On average, you're looking at $79 to $99 per user per month and then two to 5000 a year on the platform fee. Okay. Monica Hall : Okay. And this is the perfect amount of time for me if you would share the recording because I will not be the decision maker, I will be the influencer, as if it's a tool. But I. So for me, my next step would be, hey, does this look like what we're looking for? We have another system that we, we could push, I guess, from HubSpot, but it's our client success system. Okay, so are you mostly just integrated with the big boys or. No. Blake Adams : So we started, we're pretty early stage. Our go to market strategy was to, for all the major tools, integrate with the kind of two biggest players at a time. And then we do have a lot of customers who are using it on the CS side and who are using it for pre sales engineering, you know. And so we've gotten different tools that have come in wherever I. To the point now, like from a CRM perspective, we're pretty much agnostic. We can integrate with just about anything. It would just depend on what the tool is, is the short answer. Okay. Monica Hall : All right. I think it's called client success is the name of the platform. Yeah. So. Blake Adams : And then how are you leveraging that? Is that your, like your account managers or your CS writing same thing like qbrs and touch points and just writing notes back in yeah, that's client support. Monica Hall : Client support and client success. So I think the answer to the question is yes. It's sort of the account manager that, you know, the salesperson hands them off because now they're doing their setup and learning and onboarding their software and then they manage that in that system. Perfect. Because I don't think that we want to have multiple AI systems, but. So we've got the chat that is now attached to the zoom. Okay. Blake Adams : Yep. Monica Hall : It's not delivering notes to HubSpot. Right. So we're still either copying and pasting or going to search for the notes because it's nothing delivering. Blake Adams : Yeah, got it. Got it. What I will do, I'll obviously, definitely share the recording with you. I'm also going to drop a link in the chat where you can provision a pilot and kick the tires around. You can share this with any reps on your team, CS, whomever. It will understand you're all part of one organization from your domain. Okay. None of this would tie into to HubSpot. It would only join your calls, give you the pre meeting prep and post meeting summary. Then if you kick the tires around like what you see, we will do a proof of concept where we push things into HubSpot for you still totally unpaid on a pilot perspective. Okay. Monica Hall : All right. So I go to this link login, it'll attach to my Google calendar and I'll start seeing what it collects. That's right. Blake Adams : And you as a leader can actually input your team's emails. If you have two or three reps, you want to have pilot it. When you, you can put their emails in, they'll receive an invite and then you'll be able to have visibility into their coaching and metrics and all of that as well. That is one thing we didn't get into super strong that I'll show you really quickly before I send you back to your leader. Okay. In the deals view, for a sales leader, there's the, the coaching tab as well. So you've taught, you know, Oliv learned the types of things you need to do. So rather than you having to go listen to recordings and understand where a rep is good or struggling, it can actually over time culminate that, show you exactly why it's a good call, link to specific spots. But then you can also say, hey, we might need to work on whatever that, you know, area of your sales motion is and it can help you identify that more quickly. Okay. Monica Hall : Do you compete with Gong? I saw that on there. Blake Adams : Yeah. We actually, having said that, it depends on, like, if you're using gong. Some of our customers have had us integrate, push data back, pull data from. Some of our customers who were only using gong for just the recording piece have discontinued that use because it's way more expensive. Gong is super. Like I said, when we started, we integrated with it. It was never with the intent to displace it. I think it just boils down to how well people are using gone to its full functionality. What we found is a lot of customers were using it just for, like, the insights and the recording. And Oliv can replace that at a much lower price. Yeah. Monica Hall : That we were using it for a temporary time, but again, it wasn't pushing to HubSpot and. Blake Adams : Yeah, exactly. And again, I love gong. I'm a big gong fan. But as a sales manager, I never had time to go review all the calls, and it took, like, 20 to 30 minutes for me to get the summary as a seller. You know what I mean? So it was like, the difference where we don't, I wouldn't say we compete is we try to be more in the call real time rather than just give you insights after. Yeah. Monica Hall : Okay. All right, awesome. So I am definitely looking forward to having the recording. Blake Adams : Yes, I will share. Oliv will actually share all of that with you shortly. Okay. And then you just. I'll leave the ball in your court. I know we've, you know, we've had to move our time around, so I will definitely not press you. But whenever, if it is something you, you know, you provision a pilot, you kick the tires, and it's something you want to talk through, going a layer deeper, HubSpot integration, all of that, just let me know, and I'll set up time with our product team to configure that for you. Monica Hall : All right. Blake Adams : Like about two minutes. So it's nothing. Not a big lift from your team. Okay. Monica Hall : All right. I appreciate that. I'm a little full right now. That's best description. I'm full. But I do know it's a tool. So I I know it's a tool that we need just because we are working at the highest capacity that we don't. We don't have time for this. Like you're saying, jeff back in and research, and we do need some. We need a secretary. Yeah, that's. Blake Adams : It's funny you say that. I frame it to a lot of people as, like, giving all of your sellers an AI personal assistant. And that's what the tool is. You know, it's like, hey, all the admin stuff that you can replicate it can do on my behalf and let me just interact with prospects. So. Monica Hall : Yeah, yeah, that's, that's, that sounds like a nice tool, so. All right. I'll look forward to talking to you again. I will. I've already opened this. I will probably go ahead and add us to it. I just need to see what I need to, once I put myself in there, do I have to like, make fields or anything to you? Blake Adams : Do not. So everything will be coming out of the box, and again, it won't go into pub spot, so you'll have a really. Yeah, it'll be a broad stroke kind of best practices sales motion. So your pre call prep will be high level account and contact info, and then your summary will be categorized into, like, compelling event decision process objections, things like, you know, again, just kind of really standard best practices. Okay, awesome. Monica Hall : All right, awesome. Blake Adams : Yes. Monica Hall : I hope you do too. Blake Adams : Okay, bye. -- End of the transcript -- """
                        }
            user_data = {"role":"user","content":row["Prompt"],"prompt_template":prompt_template,"variables":variables}
            assistant_data = {"role":"assistant","content":row["Output_1"]}
            data_list.append(user_data)
            data_list.append(assistant_data)
            upload_data(data_list)
            data_list = []
            assistant_data = {"role":"assistant","content":row["Output_2"]}
            data_list.append(user_data)
            data_list.append(assistant_data)
            upload_data(data_list)
            break
        else:
            continue
        
        

**CODE ENDS HERE**

In [41]:
variables={"selling_company":"Oliv",
                         "buying_company":"Iti-digital",
                         "language_code":"en_us",
                         "sample_participants":"""Sample meeting participants:
    Selling team: Adam Janbolat, Ali Chaichy
    Buying team: Murali Allada, Gabriella, John Handra, Guru, David Cheng, Sekhar Boddu, Michaela Dawson""",
                        "sample_transcript":""" Sample transcript: Adam Janbolat : first off, before I forget, I send you that excel sheet Murali Allada : for the sizing Adam Janbolat : and all that. Excellent. I Murali Allada : did get that. Yeah, it's exactly the same as the original sizing. Murali Allada : I Adam Janbolat : take those numbers and plug it into our calculator. And that's how we come up with the pricing. I don't want to say spend a lot of time on it. It can be good estimations, like we've done in the past. Maybe add a little bit of growth in there, a little bit of. What do you call it? Like, how do you feel? Like about the growth? So I mean, don't be conservative. Which is Murali Allada : it's Adam Janbolat : really easy for us to say, don't be conservative, right? But I mean. We usually factor in 20%, 30%, whatever. And your case here before we did, like, 30%, and we ramped it up month after month. So anyway, just let us know whichever way you want to do that. Adam Janbolat : Email a few more people, wait for a couple of more minutes to make sure that everybody's on the call will do a quick set of introductions. And then we'll just start the conversation. And like I said, morality today. I think I talked to Ali earlier about it. Also. Happy to show a quick demo how to create the vector database. All that. But I would love to hear from you guys. Some of the use cases you're thinking about. You know what I mean? What are you thinking about in terms of generative AI. How can we help you expedite that process. Doing a POC or whatever. And then I'll go through some of the assets that we have on the website today as well. But. This is going to be a 30 minutes call. It's going to be more of a discovery call, quick demo. And then if we need to dig deeper, we can have another 1 hour call coming up in next couple of weeks. To really define the use case and start working on the POC. Murali Allada : So I can give you some examples. Adam, we're looking at it from. A very different perspective. Different teams have different requirements as the infrastructure team we are trying. To use gentle AI for discovery purposes. Basically, we've built an internal developer platform, and we want to make that easy to use for developers. So that's one perspective we are looking at that that's more of a documents search case. Basically. Alet Grew. Or maybe Gabriel, talk about analytics, data analytics. How that might be useful for us. Murali Allada : Sounds good. Let's talk it for next week. Adam Janbolat : Maybe for next week. Okay. Any good time for anybody like middle of week? First of all, I know Monday. I don't think anybody wants to be meeting on Monday. Everybody's busy Tuesday. Maybe you Guru : guys sell us. Middle to late part of next week should be good. I think morally, if Murali Allada : you can coordinate that, let's take Adam Janbolat : it from Yes. Okay. That sounds great. Thank you Adam Janbolat : Great, thank you. So I just want to make sure I got the second use case Analytics, the supply chain perspective. You think maybe for demand planning. When new items coming into the network. What would you like to see there? How do you envision using generative AI to help you with some of that. Adam Janbolat : Now if you tell me Hey. We feel like there is a POC. After this call. Maybe. We can document some of the use cases that we talked about. I might be able. To talk internally with some folks here and see hey, if we can maybe allocate some extra funds so you're not spending your own money on doing your test. Sounds good. So here's what I'll do again. I'll be sending some information here. Via an email. Some assets. I'll schedule another call with you. You tell me it sounds to me like sakar and morali. There is interest we can maybe do Friday or early next week. Whatever you want to do, we'll allocate an hour. And also I wanted to bring one other resource to the conversation. Is that reasonable morale. """,
                        "questions":""" Question: Come up with specific next steps that are actionable immediately or within a specific time for the buying team in 10 words or less. Each item should be prefixed by the task owner name followed by a hyphen. Ideally If there is some decent amount of transcript, these should not be empty. Also do 'not' add the assigned to text at the end Sample output: { "buying-team-next-steps" : [ "Murali Allada - Provide feedback on use cases for generative AI", "Murali Allada - Provide examples of requirements for different teams", "Murali Allada - Coordinate next meeting for next week" ] } Question: Come up with key takeaways that are meaningful for the buying team only in 10 words or less. Key takeaway for buying team should exclude insights about the buying team and key takeaways about the selling team that are not relevant for the buying team. Ideally if there is some decent amount of transcript, these should not be empty Sample output: { "buying-team-key-takeaways" : [ "Adam is a Data Architect at Datastax and Ali is a member of the Streaming Sales team", "Vector search is an additional feature of the paid product, Serverless Database", "The Quite Platform is currently utilizing serverless database and streaming", "Adam demonstrated a chatbot built on Astra DB and its codebase in a Jupyter notebook", "Quite team has access and can test AstraDB from their current account" ] } Question: Come up with specific next steps that are actionable immediately or within a specific time for the selling team in 10 words or less. Each item should be prefixed by the task owner name followed by a hyphen. Ideally if there is some decent amount of transcript, these should not be empty. Also do 'not' add the assigned to text at the end. In case there are no members from internal team give a one line short reason for it like 'No member joined from <company-name>' or some other relevant reason as per the situation and here <company-name> should be replaced by the selling company name. Sample output: { "selling-team-next-steps" : [ "Adam Janbolat - Send Excel sheet for sizing", "Adam Janbolat - Email all the links discussed in the call", "Adam Janbolat - Arrange next week's call with Murali and include an additional participant", "Adam Janbolat - Record Quite Platform use cases and discuss credit allocation for Vector DB trial testing with Ali and the internal team" ] } Question: Come up with key takeaways that are meaningful for the selling team only in 10 words or less. Key takeaway for selling team should include meaningful insights about the overall call and about the buying team that may include key takeaways that are not relevant for the buying team. Include topics like key pain points, objections, security and privacy concerns, key personnel to the deals, key events and timeline for the buying team. Ideally If there is some decent amount of transcript, these should not be empty. In case there are no insightful key takeaways for the selling team then use the buying-team-key-takeaways as reference and paraphrase it in a way that it is still useful for the selling team. Sample output: { "selling-team-key-takeaways" : [ "Guru, John, Gabriella, Sekhar Boddu, and Michaela hold key roles in data management, security, and platform technology at Quite", "Quite's infrastructure team uses generative AI for discovery, internal development, and document search", "The supply chain team at Quite is exploring gen AI for demand planning, forecasting, and customer profiling", "Gabriela focuses on identifying root causes for stock shortages", "Future use cases include tracking services and transportation insights, with a focus on data privacy" ] }  """,
                        "instructions":""" Go through the output schema mentioned below and for every field use the description value as guideline for how to extract the field and keep track of the values as you go through the transcript and return them in the format above """,
                        "output_schema":""" --Output Schema-- The output must be a JSON object that strictly conforms to the following JSON schema and do not output the schema itself. { "type" : "object", "properties" : { "buying-team-next-steps" : { "type" : "array", "items" : { "type" : "string" }, "description" : "Come up with specific next steps that are actionable immediately or within a specific time for the buying team in 10 words or less. Each item should be prefixed by the task owner name followed by a hyphen. Ideally If there is some decent amount of transcript, these should not be empty. Also do 'not' add the assigned to text at the end" }, "buying-team-key-takeaways" : { "type" : "array", "items" : { "type" : "string" }, "description" : "Come up with key takeaways that are meaningful for the buying team only in 10 words or less. Key takeaway for buying team should exclude insights about the buying team and key takeaways about the selling team that are not relevant for the buying team. Ideally if there is some decent amount of transcript, these should not be empty" }, "selling-team-next-steps" : { "type" : "array", "items" : { "type" : "string" }, "description" : "Come up with specific next steps that are actionable immediately or within a specific time for the selling team in 10 words or less. Each item should be prefixed by the task owner name followed by a hyphen. Ideally if there is some decent amount of transcript, these should not be empty. Also do 'not' add the assigned to text at the end. In case there are no members from internal team give a one line short reason for it like 'No member joined from <company-name>' or some other relevant reason as per the situation and here <company-name> should be replaced by the selling company name." }, "selling-team-key-takeaways" : { "type" : "array", "items" : { "type" : "string" }, "description" : "Come up with key takeaways that are meaningful for the selling team only in 10 words or less. Key takeaway for selling team should include meaningful insights about the overall call and about the buying team that may include key takeaways that are not relevant for the buying team. Include topics like key pain points, objections, security and privacy concerns, key personnel to the deals, key events and timeline for the buying team. Ideally If there is some decent amount of transcript, these should not be empty. In case there are no insightful key takeaways for the selling team then use the buying-team-key-takeaways as reference and paraphrase it in a way that it is still useful for the selling team." } }, "description" : "Categorized minutes of a meeting between a selling team and prospective buyer(s)." } -- End output schema -- """,
                        "actual_participants":""" --- Actual meeting participants: Selling team: Blake Adams Buying team: Monica Hall  """,
                        "actual_transcript":""" Actual transcript: Blake Adams : Differentiate unity to close. So not so much top of funnel but more mid funnel around meeting, execution, process adherence, CRM, data completion, things like that. So the way that we then bucket is around the actual sale or the prospect lead customer facing interaction, then the AI comes alongside the seller to help them show up fully prepared with all the information they need on the call, take notes and ensure that they get full qualification criteria. If you guys are a medic or bant or spice, whatever methodology you're using, making sure that you optimize the time, get to a yes or a no quickly and then post call culminates all of that data and can help write wrap up emails, push it back into your CRM, update necessary tools in a consistent codified fashion that also aligns to your methodology. So that's it. Very, very high level. Happy to show you around the tool and dive deeper into any of that, but that if I had to give you a 32nd overview of what all of does that tip? Monica Hall : Okay. And I'm sure I asked the question before I took the meeting, but maybe I didn't. Integration into HubSpot seamless. No problem. Blake Adams : We have a native integration. We actually use HubSpot internally and obviously we live and so great integration with HubSpot. Monica Hall : Okay. And with it. So like 90% of my, no, 99.9% of my opportunities and my sales team's opportunities are from virtual meets, just like we're having here. Right. So does Oliv take that and summarize it, put the notes into HubSpot. Can you show me what that looks like? That's a tool that I'm really looking. Yes. Blake Adams : I think you're speaking our language. So what, essentially it starts with a calendar integration, right. So it's going to see your reps, calendar, my instances. I'm a sales leader, so I've got internal meetings, interviews, all the different things. Right. So I've got workflows. But essentially our product team, during onboarding, would work with you to say for your sellers, for your CS team, for your leadership, whatever that looks like, the different types of meetings you execute, we then understand a workflow based on that. So, for instance, with our conversation, Oliv knows it did all of this, but I could change it if I needed to. Hey, this is an external meeting. We're doing discovery. So I'm going to give Blake all the things he needs for a discovery call. So in that it's bucketed in those same three areas that I mentioned, pre call, during and post, essentially in the pre call motion. Everything you see here again is my instance and my configuration, it's all completely customizable. But basically, again, Oliv learned my process and so it knows. Hey, for a first call, Blake, he typically would go to the company LinkedIn, go to the news, go to the website, look at their careers page. Maybe for Monica, he would go to her LinkedIn, check out previous roles, things like that. It then does all of this for me. Gives me a slack message, a teams message, a Google chat, whatever, ten to 20 minutes before the call, hey, you're about to meet with Monica. Here's your dossier. So you walk in prepared, you can send your warm up emails. All that's integrated back then. Any relevant information it learns can be written to HubSpot from there. Then during the call, we would map this scorecard and these fields to your HubSpot instance. Whatever is existing in your HubSpot that you want sellers filling out during a discovery call or afterwards, we can map in on the call. Again, what Oliv is doing is actually transcribing what we say in real time and reading it back with AI. So it's a little more real time than like a read AI. Fireflies, Otter, you know, some of those tools that are just recording, it's trying to actually help the seller drive. So it actually picked up HubSpot, like, right. It heard you say that. It flipped that for me. Now I'll show afterwards anything else that you said. I could use this as a talk track to actually guide me through and say, hey, I need to ask these questions to understand the tech stack. So if you're ramping reps and things like that, you can put this in front of them. Most reps live on this scorecard view because they just like to interact with the prospect. But it's been very easy to understand what I have and haven't uncovered. Then post call, I'm going, well, let's do a few just so I can show you. So are you guys a Google or a Microsoft shop? Monica Hall : I guess Google. We use Google, yeah, Google. Blake Adams : Okay, perfect. Monica Hall : We use Gmail, right? Yes, Google. Blake Adams : Google calendar. Monica Hall : HubSpot. Okay. Blake Adams : HubSpot. And then do you do your meetings because you're Google on Google Meet, or do you do zoom? Primarily zoom? Okay, and then, so you're on Google, you use zoom. And the only other one is chat. So do you guys use Google chat or slack? Monica Hall : Slack. Blake Adams : Okay. So again, just for role play sake, I'm going to have it summarized. Those would be everything that you saw in there could be customized to your sales motion. All of this would happen automatically. Oliv would get kicked out of the zoom and it would know the meeting was over. It would summarize on behalf of the reps, and then this workflow that you see is configurable to what you again, very similar to the prep, what you want your reps doing after a call. So for me, I want to see my takeaways and yours. I want to send a wrap up email. So I've taught all of my templates. It will then actually, once this summary, okay, so here comes the summary. Once this is done, it will help me write the email so I could actually let Oliv compose with my template and input the necessary things that it learned from the conversation that may be relevant. And it's also pretty smart. So like, it knows this is the first conversation. It's giving you high level information. If you had said soc two or non disclosure or anything like that, it would recommend I attach those types of documents. So it's listening for context there. And then this is where you would update your CRM bidirectionally. And this is what I wanted to show you. So it will show the rep what it has uncovered, categorized, and contextually understood based on the conversation, and changed with this nice little green Oliv. But they still can, sanity check, review whatever they want to, if they want to change anything they can. And then the minute they hit next, all of this is written into HubSpot on the opportunity in the corresponding fields. So it's essentially then guiding them step by step, making recommendations based on the context of the discovery call on how to update the CRM accurately. Monica Hall : Okay. And when it updates in the HubSpot CRM, you said it attaches it to the deal or company or you tell us. Blake Adams : So our team would work with you or your revenue operations, whomever, internally. We would then configure where to write, how to write based on what's best for your sales motion. Okay, so for our instance, it writes on the deal, right? But it could write company level contact. It can attach, we can train the model to do whatever you want it to do, essentially. Right. Okay, so that's the reps experience. Now, again, as a sales leader, you have a lot of visibility because it has access to all this data for your whole team. So just to give you, it houses all the deals, reads them from HubSpot, and then can help culminate info really quickly. It'll give you a one pager as a manager for your deal reviews, for your forecasting, for your pipeline, for your coaching, like where reps need support, because it has access to all of the data and you can really easily come in and say, you know, how, how much do we know? Is this a real deal? How much do we need to know? Maybe then you want to game plan with your rep so you can jump in, do mentions, anything like that that you want to. You can also review all of the call recordings from here, make notes, anything like that. Um, I'm trying to think if there's any other, anything else you want to know while I, while we have, because we have plenty of time. So anything else you want to dive into or explore. Monica Hall : Except I jumped off a call with my boss to jump onto this one and I told her I would come right back. But, um, so what I want to do is say, this has potential. Do you have it set up? Like, is it by seat or do you buy the platform? It's per person. Blake Adams : It's really standard SAS model per user per year and then a platform fee. So cost is variable really dependent upon integrations and configure, like customization. So how configurable do you need the instance and how many integrations do we have to build? On average, you're looking at $79 to $99 per user per month and then two to 5000 a year on the platform fee. Okay. Monica Hall : Okay. And this is the perfect amount of time for me if you would share the recording because I will not be the decision maker, I will be the influencer, as if it's a tool. But I. So for me, my next step would be, hey, does this look like what we're looking for? We have another system that we, we could push, I guess, from HubSpot, but it's our client success system. Okay, so are you mostly just integrated with the big boys or. No. Blake Adams : So we started, we're pretty early stage. Our go to market strategy was to, for all the major tools, integrate with the kind of two biggest players at a time. And then we do have a lot of customers who are using it on the CS side and who are using it for pre sales engineering, you know. And so we've gotten different tools that have come in wherever I. To the point now, like from a CRM perspective, we're pretty much agnostic. We can integrate with just about anything. It would just depend on what the tool is, is the short answer. Okay. Monica Hall : All right. I think it's called client success is the name of the platform. Yeah. So. Blake Adams : And then how are you leveraging that? Is that your, like your account managers or your CS writing same thing like qbrs and touch points and just writing notes back in yeah, that's client support. Monica Hall : Client support and client success. So I think the answer to the question is yes. It's sort of the account manager that, you know, the salesperson hands them off because now they're doing their setup and learning and onboarding their software and then they manage that in that system. Perfect. Because I don't think that we want to have multiple AI systems, but. So we've got the chat that is now attached to the zoom. Okay. Blake Adams : Yep. Monica Hall : It's not delivering notes to HubSpot. Right. So we're still either copying and pasting or going to search for the notes because it's nothing delivering. Blake Adams : Yeah, got it. Got it. What I will do, I'll obviously, definitely share the recording with you. I'm also going to drop a link in the chat where you can provision a pilot and kick the tires around. You can share this with any reps on your team, CS, whomever. It will understand you're all part of one organization from your domain. Okay. None of this would tie into to HubSpot. It would only join your calls, give you the pre meeting prep and post meeting summary. Then if you kick the tires around like what you see, we will do a proof of concept where we push things into HubSpot for you still totally unpaid on a pilot perspective. Okay. Monica Hall : All right. So I go to this link login, it'll attach to my Google calendar and I'll start seeing what it collects. That's right. Blake Adams : And you as a leader can actually input your team's emails. If you have two or three reps, you want to have pilot it. When you, you can put their emails in, they'll receive an invite and then you'll be able to have visibility into their coaching and metrics and all of that as well. That is one thing we didn't get into super strong that I'll show you really quickly before I send you back to your leader. Okay. In the deals view, for a sales leader, there's the, the coaching tab as well. So you've taught, you know, Oliv learned the types of things you need to do. So rather than you having to go listen to recordings and understand where a rep is good or struggling, it can actually over time culminate that, show you exactly why it's a good call, link to specific spots. But then you can also say, hey, we might need to work on whatever that, you know, area of your sales motion is and it can help you identify that more quickly. Okay. Monica Hall : Do you compete with Gong? I saw that on there. Blake Adams : Yeah. We actually, having said that, it depends on, like, if you're using gong. Some of our customers have had us integrate, push data back, pull data from. Some of our customers who were only using gong for just the recording piece have discontinued that use because it's way more expensive. Gong is super. Like I said, when we started, we integrated with it. It was never with the intent to displace it. I think it just boils down to how well people are using gone to its full functionality. What we found is a lot of customers were using it just for, like, the insights and the recording. And Oliv can replace that at a much lower price. Yeah. Monica Hall : That we were using it for a temporary time, but again, it wasn't pushing to HubSpot and. Blake Adams : Yeah, exactly. And again, I love gong. I'm a big gong fan. But as a sales manager, I never had time to go review all the calls, and it took, like, 20 to 30 minutes for me to get the summary as a seller. You know what I mean? So it was like, the difference where we don't, I wouldn't say we compete is we try to be more in the call real time rather than just give you insights after. Yeah. Monica Hall : Okay. All right, awesome. So I am definitely looking forward to having the recording. Blake Adams : Yes, I will share. Oliv will actually share all of that with you shortly. Okay. And then you just. I'll leave the ball in your court. I know we've, you know, we've had to move our time around, so I will definitely not press you. But whenever, if it is something you, you know, you provision a pilot, you kick the tires, and it's something you want to talk through, going a layer deeper, HubSpot integration, all of that, just let me know, and I'll set up time with our product team to configure that for you. Monica Hall : All right. Blake Adams : Like about two minutes. So it's nothing. Not a big lift from your team. Okay. Monica Hall : All right. I appreciate that. I'm a little full right now. That's best description. I'm full. But I do know it's a tool. So I I know it's a tool that we need just because we are working at the highest capacity that we don't. We don't have time for this. Like you're saying, jeff back in and research, and we do need some. We need a secretary. Yeah, that's. Blake Adams : It's funny you say that. I frame it to a lot of people as, like, giving all of your sellers an AI personal assistant. And that's what the tool is. You know, it's like, hey, all the admin stuff that you can replicate it can do on my behalf and let me just interact with prospects. So. Monica Hall : Yeah, yeah, that's, that's, that sounds like a nice tool, so. All right. I'll look forward to talking to you again. I will. I've already opened this. I will probably go ahead and add us to it. I just need to see what I need to, once I put myself in there, do I have to like, make fields or anything to you? Blake Adams : Do not. So everything will be coming out of the box, and again, it won't go into pub spot, so you'll have a really. Yeah, it'll be a broad stroke kind of best practices sales motion. So your pre call prep will be high level account and contact info, and then your summary will be categorized into, like, compelling event decision process objections, things like, you know, again, just kind of really standard best practices. Okay, awesome. Monica Hall : All right, awesome. Blake Adams : Yes. Monica Hall : I hope you do too. Blake Adams : Okay, bye. -- End of the transcript -- """
                        }

client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

sample_input_prompt = data_df.iloc[0]["Prompt"]

message = client.messages.create(
    max_tokens=4096,
    messages=[
        {
            "role": "user",
            "content": f"Given a variables list below , generate 3 variations of the values in given variables. Return the output in rc2309 json format. Note: Do not variate the language code. ---variables start---- \n {variables} \n ----variables end---",
        }
    ],
    model="claude-3-5-sonnet-20240620",
    temperature=1
)


In [40]:
from pprint import pprint
pprint(message.content[0].text)

('Here are 3 variations of the given variables, keeping the language_code '
 'unchanged:\n'
 '\n'
 'Variation 1:\n'
 "{'selling_company': 'TechPro', 'buying_company': 'GlobalSoft', "
 "'language_code': 'en_us', 'sample_participants': 'Sample meeting "
 'participants:\\n    Selling team: Sarah Johnson, Mike Chen\\n    Buying '
 "team: David Lee, Emily Watson, Robert Brown, Lisa Park, Tom Wilson', "
 '\'sample_transcript\': "Sample transcript: Sarah Johnson: Thank you all for '
 "joining today's call. I'd like to start by discussing your current "
 'challenges in data management.\\n\\nDavid Lee: Our main issue is integrating '
 'data from multiple sources efficiently.\\n\\nSarah Johnson: I see. Can you '
 "elaborate on the specific data sources you're working with?\\n\\nDavid Lee: "
 'We have data coming in from our CRM, ERP, and various IoT devices.\\n\\nMike '
 "Chen: That's a common challenge. Our platform is designed to handle diverse "
 'data sources seamlessly.\\n\\nEmily Watson: H

In [33]:
sample_input_prompt

'I am going to show you a transcript between participants from selling team "Oliv" and buying team "Iti-digital". Oliv is the selling company and Iti-digital is the buying company. Come up with answers to a set of questions. I will first provide a sample transcript, list of participants for the sample transcript, and for each question I will provide the expected output for the sample transcript. Then I will provide you with the actual meeting participants list and the actual transcript below it.  The actual transcript language code is en_us. \n\nSample meeting participants:\nSelling team: Adam Janbolat, Ali Chaichy\nBuying team: Murali Allada, Gabriella, John Handra, Guru, David Cheng, Sekhar Boddu, Michaela Dawson\n\nSample transcript:\nAdam Janbolat : first off, before I forget, I send you that excel sheet\nMurali Allada : for the sizing\nAdam Janbolat : and all that. Excellent. I\nMurali Allada : did get that. Yeah, it\'s exactly the same as the original sizing.\nMurali Allada : I\n